<a href="https://colab.research.google.com/github/hoangtheanhhp/ZaloQA/blob/bert_lstm/QASystem/run_zalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!rm -rf zalo
!test -d zalo || git clone https://github.com/hoangtheanhhp/ZaloQA zalo
import sys
if 'zalo' not in sys.path:
  sys.path+=['zalo']

Cloning into 'zalo'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 616 (delta 58), reused 54 (delta 22), pack-reused 516
Receiving objects: 100% (616/616), 94.28 MiB | 23.35 MiB/s, done.
Resolving deltas: 100% (286/286), done.
Checking out files: 100% (40/40), done.


In [0]:
%tensorflow_version 1.x
!pip install tensorflow==1.15.2
!pip install kashgari==1.1.4



TensorFlow 1.x selected.


In [0]:
DATASET_PATH = "zalo/Dataset/zalo"
SAVE_MODEL_PATH = "/content/drive/My Drive/ZaloAI/result/modelCNN_BiLSTM"
MODEL_PATH = "/content/drive/My Drive/ZaloAI/bert"

In [0]:
from zalo.QASystem.preprocess import ZaloDatasetProcessor

import kashgari
processor = ZaloDatasetProcessor()
from kashgari.embeddings import BERTEmbedding

bert_embedding = BERTEmbedding(MODEL_PATH,
                               task=kashgari.CLASSIFICATION,
                               sequence_length=128)

tokenizer = bert_embedding.tokenizer

In [0]:
# Our tokenizer already added the BOS([CLS]) and EOS([SEP]) token
# so we need to disable the default add_bos_eos setting.
bert_embedding.processor.add_bos_eos = False

In [0]:
processor.val_data = []
processor.train_data = []

In [0]:



processor.load_from_path(DATASET_PATH,"train", "train.json")
processor.load_from_path(DATASET_PATH,"val", "val.json")

data = processor.train_data
val = processor.val_data
train_x = []
train_y = []
val_x = []
val_y = []
for d in data:
  train_x.append(tokenizer.tokenize(d.get('question')) + 
                 tokenizer.tokenize(d.get('text'))[1:])
  train_y.append(str(d.get('label', False)))

for d in val:
  val_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  val_y.append(str(d.get('label', False)))

print(len(train_x), len(train_y), len(val_x), len(val_y))


100%|██████████| 3906/3906 [00:00<00:00, 719497.21it/s]


14200 14200 3906 3906


In [0]:
from tensorflow.python import keras
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model()

model.fit(train_x,
          train_y,
          val_x,
          val_y,
          epochs=100,
          batch_size=32)

model.save(SAVE_MODEL_PATH)


Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 81315072    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [0]:
from kashgari.utils import load_model

model = load_model(SAVE_MODEL_PATH)

In [0]:
processor.load_from_path("/content/zalo/Dataset/zalo", "test","test_full.json")
test = processor.test_data
test_x = []
test_y = []
for d in test:
  test_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  test_y.append(str(d.get('label')))
# evaluate the model
model.evaluate(test_x, test_y)
# print("loss: %f\n accruracy: %f\nf1_score: %f\nprecision: %f\n, recall: %f\n:" 
#       % (loss, accurany,f1_score,precision, recall))

100%|██████████| 688/688 [00:00<00:00, 164061.69it/s]


              precision    recall  f1-score   support

       False     0.8960    0.8728    0.8842      3128
        True     0.1367    0.1658    0.1498       380

    accuracy                         0.7962      3508
   macro avg     0.5163    0.5193    0.5170      3508
weighted avg     0.8137    0.7962    0.8047      3508

